In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/todo_list_dataset.csv')

In [ ]:
data.head()

,message,intent
0,Add buy groceries,add
1,Create a meeting,add
2,Add call John,add
3,Put laundry on the to-do list,add
4,Schedule a doctor's appointment,add


In [ ]:
count_intent = data['intent'].value_counts()
count_intent

add       50
view      50
delete    50
exit      50
Name: intent, dtype: int64

In [ ]:
data.duplicated().value_counts()

False    200
dtype: int64

In [ ]:
data

,message,intent
0,Add buy groceries,add
1,Create a meeting,add
2,Add call John,add
3,Put laundry on the to-do list,add
4,Schedule a doctor's appointment,add
...,...,...
195,Present the current to-dos,view
196,Display the meeting schedule,view
197,List ongoing commitments,view
198,Show me my upcoming appointments,view


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

In [ ]:
X = data['message']
y = data['intent']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [ ]:
# Convert text data into numerical features using CountVectorizer
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [ ]:
# Convert intent labels into numerical labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
# Initialize and train a Multi-Layer Perceptron (MLP) classifier
classifier = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
classifier.fit(X_train_vec, y_train_encoded)

MLPClassifier(max_iter=1000, random_state=42)

In [ ]:
# Predict intent labels on the test data
y_pred_encoded = classifier.predict(X_test_vec)

In [ ]:
# Convert numerical predictions back to intent labels
y_pred = label_encoder.inverse_transform(y_pred_encoded)

In [ ]:
# Evaluate the model
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

         add       1.00      0.92      0.96        13
      delete       1.00      1.00      1.00        12
        exit       0.91      1.00      0.95        10
        view       1.00      1.00      1.00         5

    accuracy                           0.97        40
   macro avg       0.98      0.98      0.98        40
weighted avg       0.98      0.97      0.98        40



# ***Checking overfitting by cross validation and hyperparameter tunning***

In [ ]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [ ]:
# Load the dataset
df = pd.read_csv('/content/todo_list_dataset.csv')  # Replace with the actual path to your dataset

In [ ]:
# Split the dfset into features (X) and labels (y)
X = df['message']
y = df['intent']

In [ ]:
# Split the df into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Convert intent labels into numerical labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
# Convert text df into TF-IDF features
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],
}

In [ ]:

# Initialize the MLP classifier
mlp = MLPClassifier(max_iter=1000, random_state=42)

In [ ]:
# Initialize GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(mlp, param_grid, cv=5, scoring='f1_weighted')

In [ ]:
# Fit the GridSearchCV to your data
grid_search.fit(X_train_tfidf, y_train_encoded)

GridSearchCV(cv=5, estimator=MLPClassifier(max_iter=1000, random_state=42),
             param_grid={'activation': ['relu', 'tanh'],
                         'alpha': [0.0001, 0.001, 0.01],
                         'hidden_layer_sizes': [(50,), (100,), (50, 50),
                                                (100, 50)]},
             scoring='f1_weighted')

In [ ]:
# Print the best parameters found by the grid search
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (100,)}


In [ ]:
# Get the best model from the grid search
best_mlp = grid_search.best_estimator_

In [ ]:
# Predict intent labels on the test data
y_pred_encoded = best_mlp.predict(X_test_tfidf)
y_pred = label_encoder.inverse_transform(y_pred_encoded)

In [ ]:
# Evaluate the model
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

         add       0.92      0.92      0.92        13
      delete       1.00      0.83      0.91        12
        exit       0.91      1.00      0.95        10
        view       0.83      1.00      0.91         5

    accuracy                           0.93        40
   macro avg       0.92      0.94      0.92        40
weighted avg       0.93      0.93      0.92        40



# ***Saving and Testing Model***

In [ ]:
import joblib

# Train and optimize your model

# Save the trained model, TF-IDF vectorizer, and label encoder
model_filename = 'todo_list_model.pkl'
tfidf_filename = 'tfidf_vectorizer.pkl'
encoder_filename = 'label_encoder.pkl'

joblib.dump(best_mlp, model_filename)
joblib.dump(tfidf_vectorizer, tfidf_filename)
joblib.dump(label_encoder, encoder_filename)

print(f"Model, TF-IDF vectorizer, and label encoder saved.")

Model, TF-IDF vectorizer, and label encoder saved.


In [ ]:
# Load the saved model
loaded_model = joblib.load(model_filename)

# Now you can use loaded_model to make predictions
# Example:
new_message = ["Add a new task"]
new_message_tfidf = tfidf_vectorizer.transform(new_message)
predicted_class_encoded = loaded_model.predict(new_message_tfidf)
predicted_class = label_encoder.inverse_transform(predicted_class_encoded)
print("Predicted class:", predicted_class)

Predicted class: ['add']


In [ ]:
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

# Load the saved TF-IDF model and associated preprocessing objects
model_filename = '/content/todo_list_model.pkl'
loaded_model = joblib.load(model_filename)

# Load the saved TF-IDF vectorizer and label encoder
tfidf_vectorizer = joblib.load('/content/tfidf_vectorizer.pkl')
label_encoder = joblib.load('/content/label_encoder.pkl')

# Enter a loop for continuous testing
while True:
    new_message = [input("Enter a message: ")]
    if new_message[0].lower() == 'exit':
        print("Exiting the loop.")
        break

    new_message_tfidf = tfidf_vectorizer.transform(new_message)
    predicted_class_encoded = loaded_model.predict(new_message_tfidf)
    predicted_class = label_encoder.inverse_transform(predicted_class_encoded)
    print("Predicted class:", predicted_class[0])

Enter a message: what in my list?
Predicted class: view
Enter a message: buy grocery
Predicted class: add
Enter a message: add things to list
Predicted class: add
Enter a message: remove th
Predicted class: delete
Enter a message: show today task
Predicted class: view
Enter a message: exit
Exiting the loop.


# ***use Count Vector***

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Load the dataset
data = pd.read_csv('/content/todo_list_dataset.csv')  # Replace with the actual path to your dataset

# Split the dataset into features (X) and labels (y)
X = data['message']
y = data['intent']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert intent labels into numerical labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert text data into Count Vectorized features
count_vectorizer = CountVectorizer()
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],
}

# Initialize the MLP classifier
mlp = MLPClassifier(max_iter=1000, random_state=42)

# Initialize GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(mlp, param_grid, cv=5, scoring='f1_weighted')

# Fit the GridSearchCV to your data
grid_search.fit(X_train_count, y_train_encoded)

# Print the best parameters found by the grid search
print("Best Parameters:", grid_search.best_params_)

# Get the best model from the grid search
best_mlp = grid_search.best_estimator_

# Predict intent labels on the test data
y_pred_encoded = best_mlp.predict(X_test_count)
y_pred = label_encoder.inverse_transform(y_pred_encoded)

# Evaluate the model
report = classification_report(y_test, y_pred)
print(report)

Best Parameters: {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (100,)}
              precision    recall  f1-score   support

         add       1.00      0.92      0.96        13
      delete       1.00      1.00      1.00        12
        exit       0.91      1.00      0.95        10
        view       1.00      1.00      1.00         5

    accuracy                           0.97        40
   macro avg       0.98      0.98      0.98        40
weighted avg       0.98      0.97      0.98        40



# ***Function for main code***

In [ ]:
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

# Load the saved TF-IDF model and associated preprocessing objects
model_filename = '/content/todo_list_model.pkl'
loaded_model = joblib.load(model_filename)

# Load the saved TF-IDF vectorizer and label encoder
tfidf_vectorizer = joblib.load('/content/tfidf_vectorizer.pkl')
label_encoder = joblib.load('/content/label_encoder.pkl')

# Initialize an empty list to store items
todo_list = []

# Enter a loop for continuous interaction
while True:
    new_message = input("You: ")

    if new_message.lower() == 'exit':
        print("Bot: Exiting the interaction.")
        break

    # Preprocess the input using TF-IDF vectorizer
    new_message_tfidf = tfidf_vectorizer.transform([new_message])

    # Predict intent using the loaded model
    predicted_class_encoded = loaded_model.predict(new_message_tfidf)
    predicted_class = label_encoder.inverse_transform(predicted_class_encoded)

    # Perform actions based on predicted intent
    if predicted_class[0] == 'add':
        # Extract items from the message and add them to the list
        if 'add ' in new_message:
            items = new_message.split('add ')[1].split(', ')
            todo_list.extend(items)
            print("Bot: Added to the list:", ', '.join(items))
        else:
            print("Bot: Please specify items to add.")
    elif predicted_class[0] == 'view':
        if len(todo_list) > 0:
            print("Bot: To-do list:")
            for idx, item in enumerate(todo_list, start=1):
                print(f"{idx}. {item}")
        else:
            print("Bot: The to-do list is empty.")
    elif predicted_class[0] == 'delete':
        words = new_message.split()
        ordinal_keywords = ['first', 'second', 'third', 'fourth', 'fifth', 'sixth', 'seventh', 'eighth', 'ninth', 'last']

        for keyword in ordinal_keywords:
            if keyword in words:
                index = words.index(keyword)
                if index + 1 < len(words) and words[index + 1] == 'item':
                    if keyword == 'last':
                        index = -1
                    elif keyword != 'first':
                        try:
                            num = int(keyword)
                            index = num - 1
                        except ValueError:
                            print("Bot: Invalid ordinal number.")
                            break

                    if -len(todo_list) <= index < len(todo_list):
                        deleted_item = todo_list.pop(index)
                        print(f"Bot: Deleted '{deleted_item}' from the list.")
                        break
                    else:
                        print("Bot: Index out of range.")
                        break
                else:
                    print("Bot: Please specify 'item' after the ordinal number.")
                    break
        else:
            print("Bot: Please specify the item to delete.")
    else:
        print("Bot: I'm not sure how to respond to that.")


You: add grocery, call rohan
Bot: Added to the list: grocery, call rohan
You: get my list
Bot: I'm not sure how to respond to that.
You: what in my list
Bot: To-do list:
1. grocery
2. call rohan
You: what are today task?
Bot: To-do list:
1. grocery
2. call rohan
You: put hold meeting in list
Bot: Please specify items to add.
You: add hold meeting
Bot: Added to the list: hold meeting
You: waht today task?
Bot: To-do list:
1. grocery
2. call rohan
3. hold meeting
You: remove last
Bot: Please specify 'item' after the ordinal number.
You: remove first
Bot: Please specify 'item' after the ordinal number.
You: delete first
Bot: Please specify 'item' after the ordinal number.
You: delete second
Bot: Please specify 'item' after the ordinal number.
You: exit
Bot: Exiting the interaction.


In [ ]:
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder


# Load the saved TF-IDF model and associated preprocessing objects
model_filename = '/content/todo_list_model.pkl'
loaded_model = joblib.load(model_filename)

# Load the saved TF-IDF vectorizer and label encoder
tfidf_vectorizer = joblib.load('/content/tfidf_vectorizer.pkl')
label_encoder = joblib.load('/content/label_encoder.pkl')

# Initialize an empty list to store items
todo_list = []

# Function to save the to-do list to a text file
def save_to_file(todo_list):
    with open('todo_list.txt', 'w') as file:
        for item in todo_list:
            file.write(item + '\n')

# Function to load the to-do list from the text file
def load_from_file():
    try:
        with open('todo_list.txt', 'r') as file:
            lines = file.read().splitlines()
            return lines
    except FileNotFoundError:
        return []

# Function to extract and remove an item from the list
def extract_item(words, idx):
    if idx < len(words):
        if words[idx] == "and":
            return extract_item(words, idx + 1)
        return words[idx], words[:idx] + words[idx + 1:]
    return None, words

# Load the initial list from the text file
todo_list = load_from_file()

# Enter a loop for continuous interaction
while True:
    new_message = input("You: ")


    # Preprocess the input using TF-IDF vectorizer
    new_message_tfidf = tfidf_vectorizer.transform([new_message])

    # Predict intent using the loaded model
    predicted_class_encoded = loaded_model.predict(new_message_tfidf)
    predicted_class = label_encoder.inverse_transform(predicted_class_encoded)

    # Perform actions based on predicted intent
    if predicted_class[0] == 'add':
        if 'add ' in new_message:
            items = new_message.split('add ')[1].split(', ')
            todo_list.extend(items)
            save_to_file(todo_list)  # Save the updated list
            print("Bot: Added to the list:", ', '.join(items))
        else:
            print("Bot: Please specify items to add.")
    elif predicted_class[0] == 'view':
        if len(todo_list) > 0:
            print("Bot: To-do list:")
            for idx, item in enumerate(todo_list, start=1):
                print(f"{idx}. {item}")
        else:
            print("Bot: The to-do list is empty.")

    elif predicted_class[0] == 'delete':
        words = new_message.split()
        ordinal_keywords = ['first', 'second', 'third', 'fourth', 'fifth', 'sixth', 'seventh', 'eighth', 'ninth', 'last']
        delete_indices = []

        for keyword in ordinal_keywords:
            if keyword in words:
                idx = words.index(keyword)
                item, updated_words = extract_item(words, idx + 1)
                if item is not None:
                    if item.isdigit():
                        delete_indices.append((int(item) - 1) if keyword != 'last' else -1)
                    else:
                        print("Bot: Invalid ordinal number.")
                    words = updated_words

        if delete_indices:
            deleted_items = []
            for delete_index in delete_indices:
                if -len(todo_list) <= delete_index < len(todo_list):
                    deleted_item = todo_list.pop(delete_index)
                    deleted_items.append(deleted_item)
                    save_to_file(todo_list)  # Save the updated list

            if deleted_items:
                print("Bot: Deleted {} from the list.".format(', '.join([f"'{item}'" for item in deleted_items])))
        else:
            print("Bot: Please specify the item(s) to delete.")

    elif predicted_class[0] == 'exit':
        print("Bot: Exiting the interaction.")
        break
    else:
        print("Bot: I'm not sure how to respond to that.")

You: waht in list
Bot: Please specify items to add.
You: remove list
Bot: Please specify the item(s) to delete.
You: waht in my list
Bot: Please specify items to add.
You: what in my list
Bot: To-do list:
1. hold meeting
2. call rohan
3. go for walk
4. code pratcie
5. 
6. study
7. go walk
8. washing car 
You: remove second item
Bot: Invalid ordinal number.
Bot: Please specify the item(s) to delete.
You: remove 1
Bot: Please specify the item(s) to delete.


In [ ]:
remo